# ..........................................Summer 2022 DS Internship....................................

<img src="https://victordwyer.com/wp-content/uploads/2021/09/Shopify.jpg" width=400 />

<h2> Context</h2> 

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

    1.Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
    2.What metric would you report for this dataset?
    3.What is its value?


<h2>The Data Science Process</h2>

<h3>Analysis</h3>

By using the data science process, I will determine the necessary data, clean+organize, explore, model the data and communicate what I can find based on the information provided.

In [120]:
import pandas as pd   #pandas allow us to import an excel file
import numpy as np    #math package 
import statistics

In [2]:
#let's import our data
df = pd.read_excel(r'C:\Users\mvale\OneDrive\Escritorio\Shopify\dataset2019.xlsx')

In [71]:
#First look into what the numbers say.
df[['order_amount']].describe()

,order_amount
count,5000.000000
mean,3145.128000
std,41282.539349
min,90.000000
25%,163.000000
50%,284.000000
75%,390.000000
max,704000.000000


Descriptive statistics.
What are these numbers telling us?

MAX stands out, let's analyze this next to the amount of items being purcahsed.

In [7]:
df[['order_amount','total_items']].describe()

,order_amount,total_items
count,5000.000000,5000.00000
mean,3145.128000,8.78720
std,41282.539349,116.32032
min,90.000000,1.00000
25%,163.000000,1.00000
50%,284.000000,2.00000
75%,390.000000,3.00000
max,704000.000000,2000.00000


70400 order amount doesn't sound like an average purchase.
We have 2 high values. Are they be mutually dependant?

To resolve this enigma, first step consists on doing some computing magic to see if the unit price reflects the order amount.

<h3>Science</h3>

In [14]:
#The relevant columns for this are shop_id, total_items and order_amount. let's clean the data.
dfsci = df.copy()
dfsci = dfsci [['shop_id','total_items', 'order_amount']]

In [19]:
#Let's create a variable that stores the result of the division between the amount and items to find the price 
#of each pair of sneakers.
dfsci['unit_price'] = dfsci['order_amount']/ dfsci['total_items']

In [22]:
#Mean of unit_price, max of unit price and order amount.
dfsci.describe()

,shop_id,total_items,order_amount,unit_price
count,5000.000000,5000.00000,5000.000000,5000.000000
mean,50.078800,8.78720,3145.128000,387.742800
std,29.006118,116.32032,41282.539349,2441.963725
min,1.000000,1.00000,90.000000,90.000000
25%,24.000000,1.00000,163.000000,133.000000
50%,50.000000,2.00000,284.000000,153.000000
75%,75.000000,3.00000,390.000000,169.000000
max,100.000000,2000.00000,704000.000000,25725.000000


We are getting close, here we still see some shoes pricing very high.

In [79]:
#Table to show the mean of unit price per shop to find which shop is selling gold sneakers.
dfsci.groupby(dfsci['shop_id'])['unit_price'].mean().reset_index().sort_values(by=['unit_price'], ascending = False).head()

,shop_id,unit_price
77,78,25725.0
41,42,352.0
11,12,201.0
88,89,196.0
98,99,195.0


Based on these numbers, if we take out store 78 from our data source we get closer to the information we need.

In [34]:
#Here we calculate the mean of the unit price of our data when we drop store 78
np.mean(dfsci[dfsci['shop_id'] != 78]['unit_price'])

152.47557529269278

Let's calculate the new AOV when we drop store 78.

  $$ AOV= Revenue / NumberofOrders $$

In [118]:
aov = 13461840 / 4955
print(aov)

TypeError: 'list' object is not callable

Close, but there is another store that gets our AOV very high. 

In [68]:
#Create a new variable to save our data from dropping store 78.
dfsci_1 = dfsci [(dfsci['shop_id'] != 78)]

In [69]:
#With the new data, let's see which store is causing all this noise.
dfsci_1.describe()

,shop_id,total_items,order_amount,unit_price
count,4954.000000,4954.000000,4954.000000,4954.000000
mean,49.819540,8.851029,2717.367784,152.475575
std,29.014845,116.857286,41155.996469,31.260218
min,1.000000,1.000000,90.000000,90.000000
25%,24.000000,1.000000,163.000000,132.000000
50%,50.000000,2.000000,284.000000,153.000000
75%,74.000000,3.000000,390.000000,168.000000
max,100.000000,2000.000000,704000.000000,352.000000


After reviewing this information, we have a unit price about 150 higher than the average and the order amount is still high, let's compare that with the total_items. 

In [78]:
#This table shows the MAX amount of items that were bought per shop.
dfsci_1.groupby(dfsci['shop_id'])['total_items'].max().reset_index().sort_values(by=['total_items'], ascending = False).head()

,shop_id,total_items
41,42,2000
53,54,8
84,86,6
67,68,6
66,67,6


2,000 sneakers per oder doesn't sound like an average purchase (no offense to millipedes).

Shop 42 should also be dropped since their users behaviour affects our data. 

In [112]:
#create a new variable to save the data without disrupting shops.
dfsci_2 = dfsci [(dfsci['shop_id'] != 78) & (dfsci['shop_id'] != 42)]

In [66]:
#Doublec checking we are in the right track.
np.mean(dfsci_1['order_amount'])

300.1558229655313

<h2>Results<h2>

We have made it to the final stretch, what our findings show today is that to get a relevant AOV that serves to the "normal" buyer, we would have to drop shop 78 and 42. By doing this, our pikes of information are not extremely far and we can find what does the usual buyer spends on sneakers.

Let's calculate the new AOV when we drop store 78 and 42.

  $$ AOV= Revenue / NumberofOrders $$

In [119]:
aov = 1471664/4904
print(aov)

TypeError: 'list' object is not callable

After all this computing and analyzing, now we can see that the Average Order Value is 300.09

<h1>Let's go further</h1>

Once we take a look into the numbers and start noticing numbers that are making noise, we realized there were 2 shops disrupting the rest of the information. 

This disrupting might be something good and definetely something to keep analyzing and studying to see how this kind of shops evolve in shopify's environment. 

Within the Data Science team, I would suggest getting another category for similar shops where their volume or dollar amount don't fit with the rest of the shops so we can have a full scope of how every shop is doing within our platform and better ways to serve everyone.

# ..........................................Thank you....................................